# Genbank phylogeny from COI sequences

Complete mitogenome genbank records were downloaded for 4 ant species, each from a different subfamily:
  - ***Pseudomyrmex gracilis*** (Pseudomyrmecinae)
  - ***Formica fusca*** (Formicinae)
  - ***Linepthema humile*** (Dolichoderinae)
  - ***Solenopsis invicta*** (Myrmicinae)

In [29]:
# importing everything we'll need
from Bio import SeqIO
import os, glob
import pandas as pd
import skbio.io

Now, we'll be using SeqIO to obtain the COI nucleotide sequence for each species and save it into separate files:

In [41]:
def create_dir(dir_name):
    os.makedirs(os.path.dirname(dir_name), exist_ok=True)

def extract_COI(gb_file):
    for record in SeqIO.parse(gb_file, "genbank"):
        species_name = record.annotations.get('organism').replace(" ", "_")
        filename = "./coi_seqs/{}_coi.fa".format(species_name)
        create_dir(filename)
        with open(filename, "w") as coi_file:
            for gene in record.features:
                if gene.type in ["CDS"] and gene.qualifiers.get('gene')[0] in ['COX1', 'COI']:
                    header = "{}-{}".format(species_name, gene.qualifiers.get('gene')[0])
                    sequence = gene.location.extract(record.seq) # Mas seq pode ter stop codon truncado
                    if len(sequence) % 3 == 1:
                        sequence += "AA" #Resto 1 - Precisa adicionar 'AA'
                    elif len(sequence) % 3 == 2:
                        sequence += "A" #Resto 2 - Precisa adicionar 'A'
                    coi_file.write(">{}\n{}\n".format(header, sequence))

for gb_file in glob.glob("./ant_mitogenomes/*.gb"):
    extract_COI(gb_file)

Running blastn with the COI sequences against NCBI's formicidae sequences:

**OBS:** Need to install [taxdb database](ftp://ftp.ncbi.nlm.nih.gov/blast/db/taxdb.tar.gz).

In [27]:
%%bash

# Extracting taxdb database (if necessary)
taxdb=$(pwd)/taxdb
if [[ ! -d "$taxdb" ]]; then 
    echo "taxdb dir not found. Extracting taxdb database..."
    mkdir $taxdb
    tar -C $taxdb -xaf taxdb.tar.gz
else
    echo "taxdb dir found"
fi

# Setting BLASTDB variable
echo "Setting BLASTDB variable to $taxdb"
export BLASTDB=$taxdb

# Creating directory for blast results (if necessary)
if [[ ! -d "blast_results" ]]; then 
    echo "Creating dirrectory for blast results"
    mkdir blast_results
else
    echo "blast_results directory already created"
fi

# Performing blast searches 
for coi in ./coi_seqs/*; do
    echo "Running blast search for $coi..." && 
    #blastn -query $coi -db ./blast_teste/ant_mito -out ./blast_results/$(basename $coi .fa).blast -outfmt "6 qseqid sseqid staxids sscinames stitle sacc saccver slen sstart send qseq";
    blastn -query $coi -db nr -max_target_seqs 100 -remote -entrez_query "Formicidae [Organism]" -outfmt "7 qseqid sseqid staxids sscinames stitle sacc saccver slen sstart send qseq" -out ./blast_results/$(basename $coi .fa).blast; # set max_target_seqs to higher value when running this for real
done 

taxdb dir found
Setting BLASTDB variable to /home/gabriel/Dropbox/repos/genbank_phylogeny/taxdb
blast_results directory already created
Running blast search for ./coi_seqs/Formica_fusca_coi.fa...
Running blast search for ./coi_seqs/Linepithema_humile_coi.fa...
Running blast search for ./coi_seqs/Pseudomyrmex_gracilis_coi.fa...
Running blast search for ./coi_seqs/Solenopsis_invicta_coi.fa...


**NOTE:** Blast with the `-remote` flag can take quite some time to run and is not compatible with `-taxidlist`... If hard disk space is not a problem, maybe it would be better to just download the entire nt database and run it locally...

Saving the blast results into dataframes:

In [74]:
def create_dataframe(blast_result):
    with open(blast_result) as blast7:
        df = skbio.io.read(blast7, format='blast+7', into=pd.DataFrame)
        return df

def filter_dataframe(df):
    df = df[['qseqid', 'staxids', 'sscinames', 'sacc', 'sseq']]
    df['sseq'] = df['sseq'].apply(lambda x: x.replace('-', ''))
    #df['sseqlen'] = len(df['sseq'])
    df['sseqlen'] = df.apply(lambda row: len(row.sseq), axis = 1) 
    df = df[['qseqid', 'staxids', 'sscinames', 'sacc', 'sseqlen', 'sseq']]
    return df
    
df = filter_dataframe(create_dataframe('blast_results/Formica_fusca_coi_old.blast'))
print(df)
df.to_excel("blast.xlsx")

/home/gabriel/miniconda3/lib/python3.8/site-packages/skbio/io/registry.py:547: FormatIdentificationWarning: <_io.TextIOWrapper name='blast_results/Formica_fusca_coi_old.blast' mode='r' encoding='UTF-8'> does not look like a blast+7 file
  warn("%r does not look like a %s file"
<ipython-input-74-37e3d7676a29>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sseq'] = df['sseq'].apply(lambda x: x.replace('-', ''))
<ipython-input-74-37e3d7676a29>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sseqlen

                 qseqid                staxids  \
0    Formica_fusca-COX1                  72779   
1    Formica_fusca-COX1                 208979   
2    Formica_fusca-COX1                  72779   
3    Formica_fusca-COX1                  72779   
4    Formica_fusca-COX1                  84560   
..                  ...                    ...   
495  Formica_fusca-COX1                 609739   
496  Formica_fusca-COX1                 609832   
497  Formica_fusca-COX1                 609832   
498  Formica_fusca-COX1                  72779   
499  Formica_fusca-COX1  609832;609890;2328943   

                                             sscinames      sacc  sseqlen  \
0                                        Formica fusca  LN607805     1530   
1                                       Formica selysi  KP670862     1530   
2                                        Formica fusca  FJ824419     1377   
3                                        Formica fusca  AY334398     1191   
4             